In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    #df = df[df.trip_type == 2]
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
    #df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)
    #df = df[(df.duration >= 1) & (df.duration <= 60)]
    #categorical = ['PULocationID', 'DOLocationID']
    #numerical = ['trip_distance']
    #df[categorical] = df[categorical].apply(lambda x: x.astype(str))
    return df

In [3]:
df_01 = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_01

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


Q1
Number of columns

In [4]:
len(df_01.columns) - 1

19

Q2 Standard deviation 

In [5]:
df_01.duration.std()

42.594351241920904

Q3 % of records >= 1 and <= 60

In [6]:
cleand_df_01 = df_01[(df_01['duration'] >= 1) & (df_01['duration'] <= 60)] 


In [7]:
((df_01['duration'] >= 1) & (df_01['duration'] <= 60)).mean()

0.9812202822125979

Q4 One hot encoding

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
cleand_df_01.loc[:, categorical] = cleand_df_01[categorical].astype(str)
train_dicts = cleand_df_01[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

/tmp/ipykernel_5480/1552428099.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cleand_df_01.loc[:, categorical] = cleand_df_01[categorical].astype(str)
/tmp/ipykernel_5480/1552428099.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cleand_df_01.loc[:, categorical] = cleand_df_01[categorical].astype(str)


In [9]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

Q5

In [10]:
target = 'duration'
y_train = cleand_df_01[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

/home/nkave/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.64926205038864

Q6

In [26]:
df_02 = read_dataframe('data/yellow_tripdata_2023-02.parquet')
cleand_df_02 = df_02[(df_02['duration'] >= 1) & (df_02['duration'] <= 60)] 
cleand_df_02.loc[:, categorical] = cleand_df_02[categorical].astype(str)
val_dicts = cleand_df_02[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'

y_val = cleand_df_02[target].values
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

/tmp/ipykernel_5480/959027739.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  cleand_df_02.loc[:, categorical] = cleand_df_02[categorical].astype(str)
/tmp/ipykernel_5480/959027739.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  cleand_df_02.loc[:, categorical] = cleand_df_02[categorical].astype(str)
/home/nkave/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811815790332241